In [24]:
from pathlib import Path
import pandas as pd

In [25]:
test_df = pd.read_csv(Path('Files/2023/01_Jan/202301-citibike-tripdata_1.csv'), low_memory = False)

In [26]:
test_df = test_df.dropna(how = 'any')
test_df['started_at'] = pd.to_datetime(test_df['started_at'])
test_df['ended_at'] = pd.to_datetime(test_df['ended_at'])

In [30]:
test_df['rideable_type'] = test_df['rideable_type'].map({'classic_bike': 'C', 'electric_bike': 'E'})
# test_df['member_casual'] = test_df['member_casual'].map({'member': 'M', 'casual': 'C'})
test_df['member_casual'] = test_df['member_casual'].astype(bool)

In [29]:
test_df.drop(columns = ['start_station_name', 'end_station_name'])

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,DC1CB984821DFFF7,C,2023-01-07 15:36:53.430,2023-01-07 15:39:45.406,5216.06,5145.02,40.712220,-74.010472,40.709267,-74.013247,M
1,C00CA02971557F16,C,2023-01-04 19:23:01.234,2023-01-04 19:34:57.151,5391.06,5626.13,40.719392,-74.002472,40.727791,-73.985649,M
2,C753AE5EBD8458F9,C,2023-01-20 09:22:19.894,2023-01-20 10:23:24.255,4437.01,4522.07,40.686832,-73.979677,40.689170,-73.988600,M
3,E4415A543C1972A7,C,2023-01-24 10:38:01.135,2023-01-24 10:41:40.749,5626.06,5593.01,40.724790,-73.984301,40.723356,-73.988650,M
4,BD52A87B215877C7,E,2023-01-13 10:17:38.192,2023-01-13 10:33:59.099,6955.05,6039.06,40.768292,-73.992563,40.739323,-74.008119,M
...,...,...,...,...,...,...,...,...,...,...,...
999995,58FE57A95A48C3B9,E,2023-01-16 12:56:52.115,2023-01-16 13:06:51.277,7323.09,7149.05,40.782804,-73.959659,40.774667,-73.984706,M
999996,D1E531A3C0AE3355,C,2023-01-20 06:21:24.809,2023-01-20 06:46:56.658,3192.05,4596.05,40.647379,-74.000911,40.692216,-73.984284,M
999997,3DD5E1615B32FD6F,C,2023-01-25 06:21:19.057,2023-01-25 06:49:00.240,3192.05,4596.05,40.647379,-74.000911,40.692216,-73.984284,M
999998,1AA479833FE782AE,C,2023-01-11 07:49:02.116,2023-01-11 07:58:38.117,3993.03,4010.13,40.672695,-73.954131,40.675021,-73.971115,M
